In [1]:
import pandas as pd
import re

In [2]:
scores_df = pd.read_csv("data/spreadspoke_scores.csv")
teams_df = pd.read_csv("data/teams.csv")

In [3]:
# display all NFL teams
teams_df

,team_name,team_name_short,team_id,team_id_pfr,team_conference,team_division,team_conference_pre2002,team_division_pre2002
0,Arizona Cardinals,Cardinals,ARI,CRD,NFC,NFC West,NFC,NFC West
1,Phoenix Cardinals,Cardinals,ARI,CRD,NFC,NaN,NFC,NFC East
2,St. Louis Cardinals,Cardinals,ARI,ARI,NFC,NaN,NFC,NFC East
3,Atlanta Falcons,Falcons,ATL,ATL,NFC,NFC South,NFC,NFC West
4,Baltimore Ravens,Ravens,BAL,RAV,AFC,AFC North,AFC,AFC Central
5,Buffalo Bills,Bills,BUF,BUF,AFC,AFC East,AFC,AFC East
6,Carolina Panthers,Panthers,CAR,CAR,NFC,NFC South,NFC,NFC West
7,Chicago Bears,Bears,CHI,CHI,NFC,NFC North,NFC,NFC Central
8,Cincinnati Bengals,Bengals,CIN,CIN,AFC,AFC North,AFC,AFC Central
9,Cleveland Browns,Browns,CLE,CLE,AFC,AFC North,AFC,AFC Central


In [4]:
# NFL team names from 2006 to current
teams = list(scores_df.team_home.unique())
teams_df = teams_df[teams_df.team_name.isin(teams)]
df2 = teams_df.iloc[:,[0,2]].copy()
df2.drop_duplicates(subset=['team_id'], inplace=True)
df2

,team_name,team_id
0,Arizona Cardinals,ARI
3,Atlanta Falcons,ATL
4,Baltimore Ravens,BAL
5,Buffalo Bills,BUF
6,Carolina Panthers,CAR
7,Chicago Bears,CHI
8,Cincinnati Bengals,CIN
9,Cleveland Browns,CLE
10,Dallas Cowboys,DAL
11,Denver Broncos,DEN


In [5]:
# select only regular season games
scores_df = scores_df[scores_df.schedule_playoff == False]
scores_df = scores_df.drop(columns=['schedule_playoff'])

# select 2006 season and up
scores_df = scores_df[scores_df.schedule_season >= 2006]
scores_df

,schedule_date,schedule_season,schedule_week,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
8940,09/07/2006,2006,1,Pittsburgh Steelers,28,17,Miami Dolphins,PIT,-1.5,34.5,Heinz Field,False,62.0,3.0,75,NaN
8941,09/10/2006,2006,1,Arizona Cardinals,34,27,San Francisco 49ers,ARI,-9.5,44,University of Phoenix Stadium,False,72.0,0.0,NaN,DOME
8942,09/10/2006,2006,1,Carolina Panthers,6,20,Atlanta Falcons,CAR,-4.5,39,Bank of America Stadium,False,73.0,6.0,76,NaN
8943,09/10/2006,2006,1,Cleveland Browns,14,19,New Orleans Saints,CLE,-3.0,36,FirstEnergy Stadium,False,65.0,13.0,74,NaN
8944,09/10/2006,2006,1,Detroit Lions,6,9,Seattle Seahawks,SEA,-6.0,44,Ford Field,False,72.0,0.0,NaN,DOME
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12662,12/29/2019,2019,17,Minnesota Vikings,19,21,Chicago Bears,CHI,-4.5,36,U.S. Bank Stadium,False,NaN,NaN,NaN,NaN
12663,12/29/2019,2019,17,New England Patriots,24,27,Miami Dolphins,NE,-17.0,45.5,Gillette Stadium,False,NaN,NaN,NaN,NaN
12664,12/29/2019,2019,17,New York Giants,17,34,Philadelphia Eagles,PHI,-3.5,44.5,MetLife Stadium,False,NaN,NaN,NaN,NaN
12665,12/29/2019,2019,17,Seattle Seahawks,21,26,San Francisco 49ers,SF,-3.5,47,CenturyLink Field,False,NaN,NaN,NaN,NaN


In [6]:
scores_df = scores_df.join(df2.set_index('team_id'), on='team_favorite_id')
scores_df

,schedule_date,schedule_season,schedule_week,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,team_name
8940,09/07/2006,2006,1,Pittsburgh Steelers,28,17,Miami Dolphins,PIT,-1.5,34.5,Heinz Field,False,62.0,3.0,75,NaN,Pittsburgh Steelers
8941,09/10/2006,2006,1,Arizona Cardinals,34,27,San Francisco 49ers,ARI,-9.5,44,University of Phoenix Stadium,False,72.0,0.0,NaN,DOME,Arizona Cardinals
8942,09/10/2006,2006,1,Carolina Panthers,6,20,Atlanta Falcons,CAR,-4.5,39,Bank of America Stadium,False,73.0,6.0,76,NaN,Carolina Panthers
8943,09/10/2006,2006,1,Cleveland Browns,14,19,New Orleans Saints,CLE,-3.0,36,FirstEnergy Stadium,False,65.0,13.0,74,NaN,Cleveland Browns
8944,09/10/2006,2006,1,Detroit Lions,6,9,Seattle Seahawks,SEA,-6.0,44,Ford Field,False,72.0,0.0,NaN,DOME,Seattle Seahawks
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12662,12/29/2019,2019,17,Minnesota Vikings,19,21,Chicago Bears,CHI,-4.5,36,U.S. Bank Stadium,False,NaN,NaN,NaN,NaN,Chicago Bears
12663,12/29/2019,2019,17,New England Patriots,24,27,Miami Dolphins,NE,-17.0,45.5,Gillette Stadium,False,NaN,NaN,NaN,NaN,New England Patriots
12664,12/29/2019,2019,17,New York Giants,17,34,Philadelphia Eagles,PHI,-3.5,44.5,MetLife Stadium,False,NaN,NaN,NaN,NaN,Philadelphia Eagles
12665,12/29/2019,2019,17,Seattle Seahawks,21,26,San Francisco 49ers,SF,-3.5,47,CenturyLink Field,False,NaN,NaN,NaN,NaN,San Francisco 49ers


In [7]:
# Fix naming discrepencies for LA Chargers and Rams
sd_chargers = scores_df[(scores_df.schedule_season <= 2016) & (scores_df.team_favorite_id == 'LAC')].index.values
scores_df.loc[sd_chargers, 'team_name'] = 'San Diego Chargers'
st_rams = scores_df[(scores_df.schedule_season <= 2015) & (scores_df.team_favorite_id == 'LAR')].index.values
scores_df.loc[st_rams, 'team_name'] = 'St. Louis Rams'
scores_df

,schedule_date,schedule_season,schedule_week,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,team_name
8940,09/07/2006,2006,1,Pittsburgh Steelers,28,17,Miami Dolphins,PIT,-1.5,34.5,Heinz Field,False,62.0,3.0,75,NaN,Pittsburgh Steelers
8941,09/10/2006,2006,1,Arizona Cardinals,34,27,San Francisco 49ers,ARI,-9.5,44,University of Phoenix Stadium,False,72.0,0.0,NaN,DOME,Arizona Cardinals
8942,09/10/2006,2006,1,Carolina Panthers,6,20,Atlanta Falcons,CAR,-4.5,39,Bank of America Stadium,False,73.0,6.0,76,NaN,Carolina Panthers
8943,09/10/2006,2006,1,Cleveland Browns,14,19,New Orleans Saints,CLE,-3.0,36,FirstEnergy Stadium,False,65.0,13.0,74,NaN,Cleveland Browns
8944,09/10/2006,2006,1,Detroit Lions,6,9,Seattle Seahawks,SEA,-6.0,44,Ford Field,False,72.0,0.0,NaN,DOME,Seattle Seahawks
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12662,12/29/2019,2019,17,Minnesota Vikings,19,21,Chicago Bears,CHI,-4.5,36,U.S. Bank Stadium,False,NaN,NaN,NaN,NaN,Chicago Bears
12663,12/29/2019,2019,17,New England Patriots,24,27,Miami Dolphins,NE,-17.0,45.5,Gillette Stadium,False,NaN,NaN,NaN,NaN,New England Patriots
12664,12/29/2019,2019,17,New York Giants,17,34,Philadelphia Eagles,PHI,-3.5,44.5,MetLife Stadium,False,NaN,NaN,NaN,NaN,Philadelphia Eagles
12665,12/29/2019,2019,17,Seattle Seahawks,21,26,San Francisco 49ers,SF,-3.5,47,CenturyLink Field,False,NaN,NaN,NaN,NaN,San Francisco 49ers


In [8]:
scores_df.rename(columns={'team_name': 'team_favorite_name'}, inplace=True)

In [9]:
scores_df

,schedule_date,schedule_season,schedule_week,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,team_favorite_name
8940,09/07/2006,2006,1,Pittsburgh Steelers,28,17,Miami Dolphins,PIT,-1.5,34.5,Heinz Field,False,62.0,3.0,75,NaN,Pittsburgh Steelers
8941,09/10/2006,2006,1,Arizona Cardinals,34,27,San Francisco 49ers,ARI,-9.5,44,University of Phoenix Stadium,False,72.0,0.0,NaN,DOME,Arizona Cardinals
8942,09/10/2006,2006,1,Carolina Panthers,6,20,Atlanta Falcons,CAR,-4.5,39,Bank of America Stadium,False,73.0,6.0,76,NaN,Carolina Panthers
8943,09/10/2006,2006,1,Cleveland Browns,14,19,New Orleans Saints,CLE,-3.0,36,FirstEnergy Stadium,False,65.0,13.0,74,NaN,Cleveland Browns
8944,09/10/2006,2006,1,Detroit Lions,6,9,Seattle Seahawks,SEA,-6.0,44,Ford Field,False,72.0,0.0,NaN,DOME,Seattle Seahawks
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12662,12/29/2019,2019,17,Minnesota Vikings,19,21,Chicago Bears,CHI,-4.5,36,U.S. Bank Stadium,False,NaN,NaN,NaN,NaN,Chicago Bears
12663,12/29/2019,2019,17,New England Patriots,24,27,Miami Dolphins,NE,-17.0,45.5,Gillette Stadium,False,NaN,NaN,NaN,NaN,New England Patriots
12664,12/29/2019,2019,17,New York Giants,17,34,Philadelphia Eagles,PHI,-3.5,44.5,MetLife Stadium,False,NaN,NaN,NaN,NaN,Philadelphia Eagles
12665,12/29/2019,2019,17,Seattle Seahawks,21,26,San Francisco 49ers,SF,-3.5,47,CenturyLink Field,False,NaN,NaN,NaN,NaN,San Francisco 49ers


In [10]:
# calculate home team score adjusted for the spread
adj_home_scores = scores_df.apply(lambda x: x.score_home + x.spread_favorite if x.team_home == x.team_favorite_name else x.score_home - x.spread_favorite, axis=1)
scores_df["adj_home_score"] = adj_home_scores
scores_df

,schedule_date,schedule_season,schedule_week,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,team_favorite_name,adj_home_score
8940,09/07/2006,2006,1,Pittsburgh Steelers,28,17,Miami Dolphins,PIT,-1.5,34.5,Heinz Field,False,62.0,3.0,75,NaN,Pittsburgh Steelers,26.5
8941,09/10/2006,2006,1,Arizona Cardinals,34,27,San Francisco 49ers,ARI,-9.5,44,University of Phoenix Stadium,False,72.0,0.0,NaN,DOME,Arizona Cardinals,24.5
8942,09/10/2006,2006,1,Carolina Panthers,6,20,Atlanta Falcons,CAR,-4.5,39,Bank of America Stadium,False,73.0,6.0,76,NaN,Carolina Panthers,1.5
8943,09/10/2006,2006,1,Cleveland Browns,14,19,New Orleans Saints,CLE,-3.0,36,FirstEnergy Stadium,False,65.0,13.0,74,NaN,Cleveland Browns,11.0
8944,09/10/2006,2006,1,Detroit Lions,6,9,Seattle Seahawks,SEA,-6.0,44,Ford Field,False,72.0,0.0,NaN,DOME,Seattle Seahawks,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12662,12/29/2019,2019,17,Minnesota Vikings,19,21,Chicago Bears,CHI,-4.5,36,U.S. Bank Stadium,False,NaN,NaN,NaN,NaN,Chicago Bears,23.5
12663,12/29/2019,2019,17,New England Patriots,24,27,Miami Dolphins,NE,-17.0,45.5,Gillette Stadium,False,NaN,NaN,NaN,NaN,New England Patriots,7.0
12664,12/29/2019,2019,17,New York Giants,17,34,Philadelphia Eagles,PHI,-3.5,44.5,MetLife Stadium,False,NaN,NaN,NaN,NaN,Philadelphia Eagles,20.5
12665,12/29/2019,2019,17,Seattle Seahawks,21,26,San Francisco 49ers,SF,-3.5,47,CenturyLink Field,False,NaN,NaN,NaN,NaN,San Francisco 49ers,24.5


In [11]:
# score differential
scores_df["adj_score_differential"] = scores_df.adj_home_score - scores_df.score_away
scores_df

,schedule_date,schedule_season,schedule_week,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,team_favorite_name,adj_home_score,adj_score_differential
8940,09/07/2006,2006,1,Pittsburgh Steelers,28,17,Miami Dolphins,PIT,-1.5,34.5,Heinz Field,False,62.0,3.0,75,NaN,Pittsburgh Steelers,26.5,9.5
8941,09/10/2006,2006,1,Arizona Cardinals,34,27,San Francisco 49ers,ARI,-9.5,44,University of Phoenix Stadium,False,72.0,0.0,NaN,DOME,Arizona Cardinals,24.5,-2.5
8942,09/10/2006,2006,1,Carolina Panthers,6,20,Atlanta Falcons,CAR,-4.5,39,Bank of America Stadium,False,73.0,6.0,76,NaN,Carolina Panthers,1.5,-18.5
8943,09/10/2006,2006,1,Cleveland Browns,14,19,New Orleans Saints,CLE,-3.0,36,FirstEnergy Stadium,False,65.0,13.0,74,NaN,Cleveland Browns,11.0,-8.0
8944,09/10/2006,2006,1,Detroit Lions,6,9,Seattle Seahawks,SEA,-6.0,44,Ford Field,False,72.0,0.0,NaN,DOME,Seattle Seahawks,12.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12662,12/29/2019,2019,17,Minnesota Vikings,19,21,Chicago Bears,CHI,-4.5,36,U.S. Bank Stadium,False,NaN,NaN,NaN,NaN,Chicago Bears,23.5,2.5
12663,12/29/2019,2019,17,New England Patriots,24,27,Miami Dolphins,NE,-17.0,45.5,Gillette Stadium,False,NaN,NaN,NaN,NaN,New England Patriots,7.0,-20.0
12664,12/29/2019,2019,17,New York Giants,17,34,Philadelphia Eagles,PHI,-3.5,44.5,MetLife Stadium,False,NaN,NaN,NaN,NaN,Philadelphia Eagles,20.5,-13.5
12665,12/29/2019,2019,17,Seattle Seahawks,21,26,San Francisco 49ers,SF,-3.5,47,CenturyLink Field,False,NaN,NaN,NaN,NaN,San Francisco 49ers,24.5,-1.5


In [12]:
# Create new binary column for whether the home team won (1) or lost (0) after adjusting for spread
adj_home_win = scores_df.apply(lambda x: 1 if x.adj_score_differential >= 0 else 0, axis=1)
scores_df["adj_home_win"] = adj_home_win
scores_df

,schedule_date,schedule_season,schedule_week,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,team_favorite_name,adj_home_score,adj_score_differential,adj_home_win
8940,09/07/2006,2006,1,Pittsburgh Steelers,28,17,Miami Dolphins,PIT,-1.5,34.5,Heinz Field,False,62.0,3.0,75,NaN,Pittsburgh Steelers,26.5,9.5,1
8941,09/10/2006,2006,1,Arizona Cardinals,34,27,San Francisco 49ers,ARI,-9.5,44,University of Phoenix Stadium,False,72.0,0.0,NaN,DOME,Arizona Cardinals,24.5,-2.5,0
8942,09/10/2006,2006,1,Carolina Panthers,6,20,Atlanta Falcons,CAR,-4.5,39,Bank of America Stadium,False,73.0,6.0,76,NaN,Carolina Panthers,1.5,-18.5,0
8943,09/10/2006,2006,1,Cleveland Browns,14,19,New Orleans Saints,CLE,-3.0,36,FirstEnergy Stadium,False,65.0,13.0,74,NaN,Cleveland Browns,11.0,-8.0,0
8944,09/10/2006,2006,1,Detroit Lions,6,9,Seattle Seahawks,SEA,-6.0,44,Ford Field,False,72.0,0.0,NaN,DOME,Seattle Seahawks,12.0,3.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12662,12/29/2019,2019,17,Minnesota Vikings,19,21,Chicago Bears,CHI,-4.5,36,U.S. Bank Stadium,False,NaN,NaN,NaN,NaN,Chicago Bears,23.5,2.5,1
12663,12/29/2019,2019,17,New England Patriots,24,27,Miami Dolphins,NE,-17.0,45.5,Gillette Stadium,False,NaN,NaN,NaN,NaN,New England Patriots,7.0,-20.0,0
12664,12/29/2019,2019,17,New York Giants,17,34,Philadelphia Eagles,PHI,-3.5,44.5,MetLife Stadium,False,NaN,NaN,NaN,NaN,Philadelphia Eagles,20.5,-13.5,0
12665,12/29/2019,2019,17,Seattle Seahawks,21,26,San Francisco 49ers,SF,-3.5,47,CenturyLink Field,False,NaN,NaN,NaN,NaN,San Francisco 49ers,24.5,-1.5,0
